In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [2]:
from pyspark import SparkContext

In [3]:
#default SparkContext initiation
#sc = SparkContext('local','Exercise 39') 

In [4]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("Exercise 39")\
.set("spark.executor.memory", "1g").set("spark.driver.memory", "2g")


In [5]:
sc = SparkContext(conf = conf)

In [6]:
sc

<SparkContext master=local appName=Exercise 39>

In [7]:
# Exercise 39
# Critical dates analysis
#Input: a textual csv file containing the daily value of PM10 for a set of sensors
#▪ Each line of the files has the following format
#sensorId,date,PM10 value (μg/m3)\n
#Output: an HDFS file containing one line for each sensor
#▪ Each line contains a sensorId and the list of dates with a
#PM10 values greater than 50 for that sensor


In [8]:
rdd39 = sc.textFile("input_file_39.txt")
rdd39.take(10)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5']

In [9]:
rdd39 = rdd39.map(lambda a:a.split(",") )
rdd39.take(10)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [10]:
from datetime import datetime
datetime.strptime('2016-01-02','%Y-%m-%d')


datetime.datetime(2016, 1, 2, 0, 0)

In [11]:
rdd39 = rdd39.map(lambda a: (a[0], datetime.strptime(a[1],'%Y-%m-%d') , a[1],float(a[2]) ) )
rdd39.take(10)

[('s1', datetime.datetime(2016, 1, 1, 0, 0), '2016-01-01', 20.5),
 ('s2', datetime.datetime(2016, 1, 1, 0, 0), '2016-01-01', 30.1),
 ('s1', datetime.datetime(2016, 1, 2, 0, 0), '2016-01-02', 60.2),
 ('s2', datetime.datetime(2016, 1, 2, 0, 0), '2016-01-02', 20.4),
 ('s1', datetime.datetime(2016, 1, 3, 0, 0), '2016-01-03', 55.5),
 ('s2', datetime.datetime(2016, 1, 3, 0, 0), '2016-01-03', 52.5)]

In [12]:
rdd39_proc = rdd39.map(lambda p:(p[0], p[2],p[3]) )
rdd39_proc.take(10)

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [13]:
rdd39_proc = rdd39_proc.filter(lambda p:p[2] >50 )
rdd39_proc.take(10)

[('s1', '2016-01-02', 60.2),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [14]:
rdd39_proc = rdd39_proc.map(lambda d:(d[0], d[1]))
rdd39_proc.take(10)

[('s1', '2016-01-02'), ('s1', '2016-01-03'), ('s2', '2016-01-03')]

In [15]:
rdd39_res = rdd39_proc.reduceByKey(lambda a,b: a +"," +b)

In [16]:
rdd39_res.take(10)

[('s1', '2016-01-02,2016-01-03'), ('s2', '2016-01-03')]

## Exercise 39 Bis

In [17]:
# Critical dates analysis
#Input: a textual csv file containing the daily value of PM10 for a set of sensors
#▪ Each line of the files has the following format: 
#sensorId,date,PM10 value (μg/m3)\n
#Output: an HDFS file containing one line for each sensor
#▪ Each line contains a sensorId and the list of dates with a PM10
#values greater than 50 for that sensor
#▪ Also the sensors which have never been associated with a
#PM10 values greater than 50 must be included in the result
#(with an empty set)

In [18]:
rdd39bis = sc.textFile("input_file_40.txt")
rdd39bis.take(10)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5',
 's3,2016-01-03,12.5']

In [19]:
rdd39bis = rdd39bis.map(lambda b:b.split(","))
rdd39bis.take(10)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5'],
 ['s3', '2016-01-03', '12.5']]

In [20]:
rdd39bis = rdd39bis.map(lambda a: (a[0], a[1],float(a[2]) ) )
rdd39bis.take(10)

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5),
 ('s3', '2016-01-03', 12.5)]

In [21]:
rdd39bis_ok = rdd39bis.filter(lambda f: f[2]>50)
rdd39bis_not = rdd39bis.filter(lambda f: not f[2]>50)


In [22]:
rdd39bis_ok.take(10)

[('s1', '2016-01-02', 60.2),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [23]:
rdd39bis_not.take(10)

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s2', '2016-01-02', 20.4),
 ('s3', '2016-01-03', 12.5)]

In [24]:
rdd39bis_test = rdd39bis.map(lambda f:(f[0],f[1],f[2] if f[2]>50 else 0 ))
rdd39bis_test.take(10)

[('s1', '2016-01-01', 0),
 ('s2', '2016-01-01', 0),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 0),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5),
 ('s3', '2016-01-03', 0)]

In [25]:
rdd39bis_test = rdd39bis_test.map(lambda y:(y[0], y[1] if y[2] != 0 else "" ))
rdd39bis_test.take(10)

[('s1', ''),
 ('s2', ''),
 ('s1', '2016-01-02'),
 ('s2', ''),
 ('s1', '2016-01-03'),
 ('s2', '2016-01-03'),
 ('s3', '')]

In [26]:
rdd39bis_test_ok = rdd39bis_test.filter(lambda f: not f[1]=="")
rdd39bis_test_not = rdd39bis_test.filter(lambda f: f[1]=="")

In [27]:
rdd39bis_test_ok.take(10)

[('s1', '2016-01-02'), ('s1', '2016-01-03'), ('s2', '2016-01-03')]

In [28]:
rdd39bis_test_ok = rdd39bis_test_ok.reduceByKey(lambda h,j: h +"," +j)
rdd39bis_test_ok.take(10)

[('s1', '2016-01-02,2016-01-03'), ('s2', '2016-01-03')]

In [31]:
rdd39bis_test_not.take(10)


[('s1', ''), ('s2', ''), ('s2', ''), ('s3', '')]

In [34]:
rdd39bis_test_ok.distinct().take(10)

[('s1', '2016-01-02,2016-01-03'), ('s2', '2016-01-03')]

In [35]:
rdd39bis_test_not.distinct().take(10)

[('s1', ''), ('s2', ''), ('s3', '')]

In [37]:
rdd39bis_test_ok.union(rdd39bis_test_not).distinct().take(10)

[('s2', '2016-01-03'),
 ('s1', ''),
 ('s3', ''),
 ('s1', '2016-01-02,2016-01-03'),
 ('s2', '')]

In [39]:
rdd39bis_test_ok.cartesian(rdd39bis_test_not).take(100)

[(('s1', '2016-01-02,2016-01-03'), ('s1', '')),
 (('s1', '2016-01-02,2016-01-03'), ('s2', '')),
 (('s1', '2016-01-02,2016-01-03'), ('s2', '')),
 (('s1', '2016-01-02,2016-01-03'), ('s3', '')),
 (('s2', '2016-01-03'), ('s1', '')),
 (('s2', '2016-01-03'), ('s2', '')),
 (('s2', '2016-01-03'), ('s2', '')),
 (('s2', '2016-01-03'), ('s3', ''))]